In [ ]:
import geopandas as gpd                # to read vector files (based on ogr)
import pandas as pd                    # to read .txt and manipulate data
from rasterstats import zonal_stats    # to conduct calculations on our rasters
import rasterio                        # To open and up .tif (Based on Gdal)
from rasterio.plot import show         # To visualise .tif opened up by rasterio

## Set paths to files and read them using geopandas (gpd) and rasterio

In [ ]:
dsm_file = 'DSM_LondonCity_1m.tif'
shp_file = 'drone_pos_32631.gml'

# to open a shapefile in Geopandas use gpd.read_file(Path_to_file)
drones = gpd.read_file(shp_file)

# to open a .tif file in rasterio use gpd.open(Path_to_file)
dsm = rasterio.open(dsm_file)

# print to check that the coordinate system for our files matches!
# If True, we are good to go.
drones.crs == dsm.crs 

#### Not sure what the warning actually mean. Others seems to have got this issue, but it does not seem to affect the preformance that much

In [ ]:
# print to se what is inside drones. .head() prints out only 5 first rows. .tail() will print last 5
drones.info()

drones.head()

## Use .drop(columns=(' ') to remove columns we do not need

In [ ]:
drones = drones.drop(columns=(['fid', 'left','top','right','bottom']))
drones.head()

# Plot DSM and points

In [ ]:
show(dsm)
# With Show() you can also add titles and change colors etc. like this below
#show(dsm_show, title='DSM London', cmap = 'terrain_r') 

In [ ]:
# To plot the dronepoints just use .plot()
# This will probably not say that much as you just see the points
drones.plot(figsize = (10,10), markersize = 10, column = 'id', cmap = 'rainbow')
show(dsm)


## use geopandas built-in function buffer to buffer around points. 

In [ ]:
# Settings for buffer
style = 3 # 1 circle 3 square
radius = 100. # for float

# Buffer around points using 'geometry' colums. Call the new buffered geometry 'buff'
drones['buff'] = drones['geometry'].buffer(radius, cap_style=style)
drones

## Visualise buffers on Dsm


In [ ]:
ax = drones['buff'].plot(facecolor = 'none' , edgecolor='white', figsize = (15,8)).axis('off')
show(dsm)

# Try changing to another buffer size and see the *

## Calculate zonal statistics for each buffer zone using rasterstats zonal_stats()

### Furter information on rasterstats https://pythonhosted.org/rasterstats/


In [ ]:
# zonal_stats can calculate a lot of things. just chose the ones you want
# if stats is not specified as zonal_stats(shapefile, raster), count, min, max and mean is calculated  

# .loc[0] to select row index 0, in this case to not to have zonal_statistics printed for all rows

zonal_stats(drones.loc[0,['buff']],dsm_file, 
            stats = ['count', 'min', 'max', 'mean', 'sum', 'std', 'median', 
                     'majority', 'minority', 'unique', 'range', 'nodata', 'nan'])


## Join zonal_statistics to drone dataframe

In [ ]:
# Create a new dataframe (drones_zonal_stats) 
# .join() join dataframe with other dataframe
# zonal_stats(drones['buff'], dsm) Calculates min, max, mean, count for each buffer zone from the raster

drones_zonal_stats = drones.join(pd.DataFrame(zonal_stats(drones['buff'],dsm_file)))

drones_zonal_stats.head()


## Plot mean height from zonal statistics. Change geometry column to buffer instead of point


In [ ]:
drones_zonal_stats.set_geometry('geometry').plot(
    column = 'mean', cmap = 'coolwarm', figsize = (10,6), legend = True).axis('off')
show(dsm)

## Print to .txt using specified columns

In [ ]:
# sep = sets separator
# float_format sets numbers of decimals

# .to_csv('filename.txt', sep = 'separator of choice', float_format = '%.numberf')

drones_zonal_stats.set_index('id')[['mean','min','max']].to_csv('droneheight_jupyter.txt',
                                                                sep = '\t',float_format = '%.3f')

pd.read_csv('droneheight_jupyter.txt', sep = '\t', index_col = 'id')

## Create a shapefile or other filetype from zonal statistics using geopandas

In [ ]:
gpd.GeoDataFrame(drones_zonal_stats[['mean','min','max','buff']], geometry='buff').to_file('drone_zonal_stats.shp')

gpd.read_file('drone_zonal_stats.shp').plot(column='mean').axis('off')
show(dsm)

# Here is a shortened version of the same script

In [ ]:
import geopandas as gpd
import pandas as pd
from rasterstats import zonal_stats


# Inputs
dsm = 'DSM_LondonCity_1m.tif'
vectorlayer = 'drone_pos_32631.gml'
drones = gpd.read_file(vectorlayer)

# Set radius and syle for buffer 
style = 3 # 1 circle 3 square
radius = 100.

# Buffer point
drones['buff'] = drones['geometry'].buffer(radius, cap_style = 3)

# Calculate mean min max count (zonal statistics using the created buffer")
drones_zonal_stats = drones.join(pd.DataFrame(zonal_stats(drones['buff'],dsm)))

# Print to txt file for columns: mean, min , max with tab-seperator
drones_zonal_stats.set_index('id')[['mean','min','max']].to_csv('droneheight_gpd_rasterio.txt', sep = '\t')
